## Ensamblo mis dos mejores modelos

In [1]:
library(data.table)

ensamblar_y_generar_submit <- function(rutas_modelos,
                                      out_file,
                                      envios  = 11000,
                                      id_col  = "numero_de_cliente",
                                      prob_col = "prob") {
  # --------------------------
  # 1) Leer predicciones
  # --------------------------
  if (length(rutas_modelos) < 2) {
    stop("Necesitás al menos 2 archivos de predicción para hacer ensamble.")
  }

  pred_list <- lapply(rutas_modelos, fread)

  # --------------------------
  # 2) Chequeos básicos
  # --------------------------
  # todas las tablas deben tener la columna de ID y prob
  for (i in seq_along(pred_list)) {
    dt <- pred_list[[i]]
    if (!(id_col %in% names(dt))) {
      stop("El archivo ", rutas_modelos[i], " no tiene la columna ID: ", id_col)
    }
    if (!(prob_col %in% names(dt))) {
      stop("El archivo ", rutas_modelos[i], " no tiene la columna de probabilidad: ", prob_col)
    }
  }

  # --------------------------
  # 3) Alinear por numero_de_cliente
  # --------------------------
  # Empiezo por el primero
  base <- pred_list[[1L]][, .(numero_de_cliente = get(id_col),
                              prob_1 = get(prob_col))]

  setkey(base, numero_de_cliente)

  if (length(pred_list) > 1L) {
    for (i in 2:length(pred_list)) {
      dt_i <- pred_list[[i]][, .(numero_de_cliente = get(id_col),
                                 prob_i = get(prob_col))]

      setkey(dt_i, numero_de_cliente)

      # merge 1 a 1 por ID
      base <- base[dt_i, nomatch = 0]  # inner join: solo IDs comunes

      # renombrar la nueva columna de prob para que no pise
      setnames(base, "prob_i", paste0("prob_", i))
    }
  }

  # Ahora base tiene: numero_de_cliente, prob_1, prob_2, ..., prob_k
  prob_cols <- grep("^prob_", names(base), value = TRUE)

  if (length(prob_cols) < 2L) {
    stop("No se generaron múltiples columnas de probabilidad. Revisar merge.")
  }

  # --------------------------
  # 4) Promedio fila a fila
  # --------------------------
  base[, prob_ensamble := rowMeans(.SD), .SDcols = prob_cols]

  # --------------------------
  # 5) Ordenar y quedarnos con top N
  # --------------------------
  setorder(base, -prob_ensamble)

  # cortar a los N envíos
  top_envios <- base[1:envios, .(numero_de_cliente)]

  # --------------------------
  # 6) Guardar CSV estilo submit: solo IDs, sin header
  # --------------------------
  fwrite(top_envios,
         file      = out_file,
         col.names = FALSE,
         row.names = FALSE,
         quote     = FALSE)

  cat("✔ Ensamble generado y guardado en:", out_file,
      " | Total enviados:", nrow(top_envios), "\n")

  invisible(top_envios)
}


In [2]:
setwd("/home/vickydiliscia/buckets/b1/exp/")
dir.create("Ensamble-C2020-C2050")


In [3]:
setwd("/home/vickydiliscia/buckets/b1/exp/Ensamble-C2020-C2050")


In [5]:
rutas <- c(
  "/home/vickydiliscia/buckets/b1/exp/APO2020-202106/prediccion_final.txt",
  "/home/vickydiliscia/buckets/b1/exp/APO2050-202106/prediccion_final.txt"
)

out_file <- sprintf("submit_ensamble_c2020_c2050.csv")

ensamblar_y_generar_submit(
  rutas_modelos = rutas,
  out_file      = out_file,
  envios        = 11000,           # top-N
  id_col        = "numero_de_cliente",
  prob_col      = "prob"
)


✔ Ensamble generado y guardado en: submit_ensamble_c2020_c2050.csv  | Total enviados: 11000 
